<a href="https://colab.research.google.com/github/TheOneTrueGuy/202-Llama/blob/main/1Gemini_3cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Digital daydreaming in latent space with CLIP Interogator and Stable Diffusion

In [ ]:
# reimplementing this to use google gemini API instead of clip interogator. Image generation will be handled with SD3-medium

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install --upgrade accelerate #transformers
!pip install omegaconf
!apt-get install rsync # for better file transfer progress?
!pip install noise
!pip install -q -U google-generativeai

In [ ]:
#@title Mount Google Drive
import os
from google.colab import drive

drive.mount('/content/drive')
results_dir = '/content/drive/MyDrive/clip-loop'
os.makedirs(results_dir, exist_ok=True)

# fetch image_to_image.py
#!git clone https://github.com/TheOneTrueGuy/DigitalDayDream.git
#!cp DigitalDayDream/noiser.py /content/

print(f"\nResults will be saved to {results_dir}")


In [ ]:
!apt install imagemagick
!pip install opencv-python
!cp /content/drive/MyDrive/stub_materials/broca1.txt /content/
!cp /content/drive/MyDrive/stub_materials/bio1.txt /content/
!cp /content/drive/MyDrive/stub_materials/command.txt /content/
!cp /content/drive/MyDrive/stub_materials/noiser.py /content/
!cp /content/drive/MyDrive/stub_materials/trim4.py /content/trim4.py
!cp /content/drive/MyDrive/stub_materials/mirrorhalf.py /content/mirrorhalf.py
!cp /content/drive/MyDrive/stub_materials/stereogram.py /content/
!cp /content/drive/MyDrive/stub_materials/scroll4.py /content/
!cp /content/drive/MyDrive/stub_materials/1julien3.sh /content/
!cp /content/drive/MyDrive/stub_materials/gauss_noiser.py /content/
!cp /content/drive/MyDrive/stub_materials/moto*.txt /content/

In [ ]:
#@title upload start image 512x512 will be quick and look good

import os

from google.colab import files
uploaded = files.upload()
name=uploaded.keys()
name = list(name)[0]
print(name)
os.system(f"convert {name} -resize 512x512! st.png")

In [ ]:
os.system(f"convert st.jpg -resize 512x512! st.png")

In [ ]:
# run this first then make anim in next cell

import torch
#from diffusers import StableDiffusionXLImg2ImgPipeline
#from diffusers import StableDiffusion3Pipeline
from diffusers.models import AutoencoderKL

#from diffusers import StableDiffusionImg2ImgPipeline
from diffusers.utils import load_image
from transformers import AutoTokenizer # for use with small sd
#from diffusers import DiffusionPipeline
import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image, ImageOps, ImageChops, ImageFilter

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, [False])

torch.cuda.empty_cache()
# check to see if pipe is loaded, if not load it otherwise skip loading pipe

#if pipe is None:

#from google.colab import userdata
#hf_auth=userdata.get('HF_TOKEN')
#print(hf_auth)
hf_auth="your HF auth here"
#!huggingface-cli login

#from diffusers import StableDiffusion3Pipeline
from diffusers import StableDiffusion3Img2ImgPipeline
#pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers",
#                                                torch_dtype=torch.float16, use_auth_token=hf_auth)

pipe = StableDiffusion3Img2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers",
                                                torch_dtype=torch.float16, token=hf_auth,  text_encoder_3=None, tokenizer_3=None)

#pipe = StableDiffusionImg2ImgPipeline.from_single_file(
#  "deliberatexp.safetensors",
#  torch_dtype=torch.float16,
#)

#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "runwayml/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16
#) # can't use LoRa??

#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16
#) # can't use LoRa??


#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "/content/drive/MyDrive/AI/models/stable-diffusion-xl-refiner-1.0/sd_xl_refiner_1.0.safetensors",
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)

#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "/content/drive/MyDrive/AI/models/deliberate_v3.safetensors",
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)

#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "sdxlUnstableDiffusers_v9DIVINITYMACHINEVAE.safetensors",
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)
#vae = AutoencoderKL.from_single_file("sdxl_vae.safetensors")


#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "segmind/small-sd", torch_dtype=torch.float16
#) # error: 'NoneType' object has no attribute 'tokenize'
#I get it, need to supply a tokenizer before calling "generate, cool"
#tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
#pipe.tokenizer=tokenizer

#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
#)
# use this one for loras

#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "sdxlUnstableDiffusers_v9DIVINITYMACHINEVAE.safetensors", torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)
# sdxlUnstableDiffusers_v9DIVINITYMACHINEVAE.safetensors
#cyborg
#chillpixel/starlight-animated-sdxl

#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "", torch_dtype=torch.float16
#)
# stabilityai/stable-diffusion-xl-base-1.0 # use with load lora

#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "/content/drive/MyDrive/AI/models/deliberate_v3.safetensors", torch_dtype=torch.float16, repo_type="local"
#)

#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file("/content/drive/MyDrive/AI/models/deliberate_v3.safetensors")


#vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "sd_xl_refiner_1.0.safetensors", vae=vae,
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)

#text_encoder = CLIPTextModel.from_pretrained('clip_g.safetensors')
#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(<checkpoint path>, text_encoder=text_encoder)


#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "sd3_medium.safetensors", #text_encoder=text_encoder, #vae=vae,
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)

#pipe = StableDiffusionXLImg2ImgPipeline.from_single_file(
#  "sd3_medium_incl_clips.safetensors", # text_encoder=text_encoder, #vae=vae,
#  torch_dtype=torch.float16, use_safetensors=True, repo_type="local"
#)

#from diffusers import EDMDPMSolverMultistepScheduler
#pipe.scheduler = EDMDPMSolverMultistepScheduler()

#pipe = DiffusionPipeline.from_pretrained(
#            "playgroundai/playground-v2.5-1024px-aesthetic",
#            torch_dtype=torch.float16,
#            variant="fp16",
#        ).to("cuda")

      # # Optional: Use DPM++ 2M Karras scheduler for crisper fine details
      # from diffusers import EDMDPMSolverMultistepScheduler
      # pipe.scheduler = EDMDPMSolverMultistepScheduler()

pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()

# time for local loading of model: 2:30
# time for remote loading of model: 1m 0s

def generate_phrase(word_string, spacing, added_word):
    words = word_string.split()
    result = []
    lenwo = len(words)

    for yn in range(lenwo):
        result.append(words[(yn) % lenwo])
        if yn % spacing == spacing - 1:
            result.append(added_word)

    phrase = ' '.join(result)
    return phrase

from collections import deque

class Stack:
    def __init__(self):
        self.stack = deque(maxlen=9)
        self.count = 0

    def push(self, string1, string2, string3):
        self.stack.append([string1, string2, string3])
        self.count += 1
        if self.count>19: self.count=19

    def get_stack(self):
        return list(self.stack)

    def get_item(self, index):
        if index >= 0 and index < len(self.stack):
          return self.stack[index]
        else:
          return ["error", "error", "error"]

stacky=Stack()



In [ ]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
#@title Create animation!

start_image_path = "/content/st.png" #@param {type:"string"}
style_prompt = "experimental medium, asmr, " #@param {type:"string"}
dir_name = "Gemini_cycle_2" #@param {type:"string"}
max_frames = 800 #@param {type:"integer"}

import datetime
import os
import torch
import numpy as np
from PIL import Image
import time
import random
from transformers import CLIPTokenizer
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32") # need to fix for correct token length but sorta works

#had to remove motion control for github listing

#file2 = open("moto1g.txt")
#lines2 = file2.read().replace("\n", "")
#file2.close()
## use at least one following lines if you don't have any moto files and comment out above 3 lines
#lines2="zi zr zi zr zi zr zi sr zt sl zi sr zi sl zi sr zi sl zi sr zi sl sl sl sr sr sr zi zr zi sd sd zr sd sd nm su su su"
#lines2=" ".join(["zi"] * 22) + " " + " ".join(["tr"] * 15) +  " " + " ".join(["nm"] * 9) #loopmoto
#lines2=lines2+" ".join(["zi"] * 20) + " " + " ".join(["td"] * 15) +  " " + " ".join(["zi"] * 22) +  " " + " ".join(["nm"] * 12)
#lines2=" ".join(["zi"] * 18) + " " + " ".join(["zr"] * 5) + " " + " ".join(["nm"] * 20) + " " + " ".join(["zr"] * 6) #loopmoto
#lines2=" ".join(["tu"] * 30) + " " + " ".join(["tl"] * 30) + " " + " ".join(["zi"] * 50) + " ".join(["td"] * 30) + " " + " ".join(["tr" ] * 30) + " " + " ".join(["zi"] * 50) #loopmoto
moto=lines2.split(" ")
lenmo=len(moto)

nois=0.8

rond=random.randint(1, 10000000)
os.makedirs(dir_name, exist_ok=True)
dialog = open("dialog.txt", "a", encoding="utf-8")
dialog.write("random seed:"+str(rond) + "\n")
primary="This is a visual representation of your current thinking. Using only visually descriptive terms describe the next thought that occurs to you based on it."
for xn in range(0, max_frames, 1):
  im = Image.open(start_image_path if xn == 0 else "st.png").convert("RGB")
  im = im.resize((512,512))

  buffered = BytesIO()
  im.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue()).decode()
  #cliprompt = ci.interrogate_classic(im)
  #numtokens=len(tokenizer.tokenize(cliprompt))
  # Pass the base64 encoded image to the model
  gemprompt=model.generate_content([primary, {"image": {"data": img_str}}])


  # Check if the response contains any parts
  if gemprompt.parts:
    if style_prompt: #replace this with output from gemini vision model
        cliprompt = gemprompt.text # Access the text content of the response
        cliprompt = style_prompt + cliprompt
    else:
        cliprompt = gemprompt.text #sometimes I add unnecessary steps just for the hell of it.

    gemprompt = cliprompt
  else:
    # Handle the case where the response is blocked
    print(f"{xn:04d}: Response blocked by safety filter\n")
    dialog.write(f"{xn:04d}: Response blocked by safety filter\n")
    continue # Skip to the next iteration

  # how to shorten gemprompt length in tokens so they are less than 77?
  gem_token_len = len(tokenizer.tokenize(gemprompt))
  if gem_token_len > 70: # extra 7 just to be sure
    gemprompt = tokenizer.decode(tokenizer.encode(gemprompt)[:70])


  print(f"{xn:04d}: {gemprompt}\n")
  #dialog.write(f"{xn:04d}: {cliprompt},\n")
  dialog.write(f"{xn:04d}: {gemprompt},\n")

  image = pipe(
      prompt=gemprompt,
      image=im,
      strength=0.6,
      guidance_scale=8.2,
      num_inference_steps=16
      ).images[0]

  image.save("st.png")
  #os.system("python noiser.py -p prev.png -n " +str(int(xn*2.8)))
  image.save(f"{dir_name}/{xn:04d}.png")

  # and you will have to compile the video with ffmpeg or something
  #next 2 lines apply motion distortifrom moto*.txt motion fileson

  print(str(xn) + " of " + str(max_frames) + " motion:" + moto[xn%lenmo] )

  #coman="bash 1julien3.sh st.png " + moto[xn%lenmo]

  #would like output from this bash command captured and printed out
  #os.system(coman)


  # be sure to add extra noise if zooming out i.e. moto1f

  #os.system(f"python gauss_noiser.py st.png {nois} st.png")

dialog.close()

# zip animation frames and put in Google Drive
year_month_day = datetime.datetime.now().strftime("%Y_%m_%d")
zip_name = f"{year_month_day}_{dir_name}.zip"

!mv dialog.txt $dir_name/
!zip -r $zip_name $dir_name/
!cp $zip_name $results_dir

print(f"Animation frames exported to {results_dir}/{zip_name}")


 painting : photorealistic image rendered giza : Uxmal  desert, : jungle,  painting : rendered image  drawing : black and white photo : colorful stucco  egyptian men : elegant parrots on, : on surrounded by techno jungle,  dreadlocks : feathers skull:skully skull:white turnip apples:crystals green:green and red fractal:neon liquid paint splash psychedelic:asymmetric fractal kaleidoscop spiral:maze puzzle
 woman:flower headphones on:mechanical tendrils a close up : wide angle a close up of a person wearing headphones:wide angle view of a factory

In [ ]:
#run this cell after starting cell above if you don't want to burn
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()

In [ ]:
year_month_day = datetime.datetime.now().strftime("%Y_%m_%d")
zip_name = f"{year_month_day}_{dir_name}.zip"
!mv dialog.txt $dir_name/
!zip -r $zip_name $dir_name/
!cp $zip_name $results_dir


In [ ]:
#working scrap section, sorry
file2 = open("moto1.txt")
lines2 = file2.read().replace("\n", " ")
file2.close()
moto=lines2.split(" ")
lenmo=len(moto)